In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import udf, struct, col
from pyspark.sql.types import *

In [3]:
'''
Got java.lang.OutOfMemoryError: Java heap space error. This error occurs when the Java Virtual Machine (JVM) running Spark 
does not have enough memory to allocate for the operation we are performing.
spark.driver.memory property is set to "4g", which allocates 4 gigabytes of memory for the driver
'''
spark= SparkSession.builder.appName("Data Analysis on user data").config("spark.driver.memory", "4g").getOrCreate()

In [4]:
user_df = spark.read.json("C:\Supriyaa-spark-notes\yelp\yelp_academic_dataset_user.json")

In [5]:
user_df.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [6]:
user_df.columns

['average_stars',
 'compliment_cool',
 'compliment_cute',
 'compliment_funny',
 'compliment_hot',
 'compliment_list',
 'compliment_more',
 'compliment_note',
 'compliment_photos',
 'compliment_plain',
 'compliment_profile',
 'compliment_writer',
 'cool',
 'elite',
 'fans',
 'friends',
 'funny',
 'name',
 'review_count',
 'useful',
 'user_id',
 'yelping_since']

In [7]:
nrow=user_df.count()
ncol=len(user_df.columns)
print("Dataset Shape:",'(', nrow,',',ncol,')')

Dataset Shape: ( 1987897 , 22 )


In [8]:
user_df.dtypes

[('average_stars', 'double'),
 ('compliment_cool', 'bigint'),
 ('compliment_cute', 'bigint'),
 ('compliment_funny', 'bigint'),
 ('compliment_hot', 'bigint'),
 ('compliment_list', 'bigint'),
 ('compliment_more', 'bigint'),
 ('compliment_note', 'bigint'),
 ('compliment_photos', 'bigint'),
 ('compliment_plain', 'bigint'),
 ('compliment_profile', 'bigint'),
 ('compliment_writer', 'bigint'),
 ('cool', 'bigint'),
 ('elite', 'string'),
 ('fans', 'bigint'),
 ('friends', 'string'),
 ('funny', 'bigint'),
 ('name', 'string'),
 ('review_count', 'bigint'),
 ('useful', 'bigint'),
 ('user_id', 'string'),
 ('yelping_since', 'string')]

In [9]:
user_df.describe().show()

+-------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+-----------------+------------------+-------------------+-----------------+------------------+-----------------+------------------+--------------------+------------------+------------+------------------+-----------------+--------------------+-------------------+
|summary|     average_stars|   compliment_cool|    compliment_cute|  compliment_funny|    compliment_hot|    compliment_list|    compliment_more|   compliment_note|compliment_photos|  compliment_plain| compliment_profile|compliment_writer|              cool|            elite|              fans|             friends|             funny|        name|      review_count|           useful|             user_id|      yelping_since|
+-------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-

In [10]:
import time
# Start the timer
start_time = time.time()

#print distinct values of each column
from pyspark.sql.functions import col


# Iterate over each column in the DataFrame
for column in user_df.columns:
    column_df = user_df.select(col(column)).distinct()
    # Execute the action and print the result
    distinct_values = column_df.count()
    print(f"Distinct values of {column}:", distinct_values)
    
# Stop the timer
end_time = time.time()

# Calculate the execution time
execution_time = end_time - start_time

print('\n')

# Print the execution time
print("Execution Time:", execution_time, "seconds")

Distinct values of average_stars: 401
Distinct values of compliment_cool: 1604
Distinct values of compliment_cute: 308
Distinct values of compliment_funny: 1604
Distinct values of compliment_hot: 1312
Distinct values of compliment_list: 193
Distinct values of compliment_more: 352
Distinct values of compliment_note: 1010
Distinct values of compliment_photos: 1018
Distinct values of compliment_plain: 1663
Distinct values of compliment_profile: 345
Distinct values of compliment_writer: 822
Distinct values of cool: 4532
Distinct values of elite: 1356
Distinct values of fans: 691
Distinct values of friends: 1091999
Distinct values of funny: 3757
Distinct values of name: 132361
Distinct values of review_count: 1973
Distinct values of useful: 5349
Distinct values of user_id: 1987897
Distinct values of yelping_since: 1980183


Execution Time: 195.28213381767273 seconds


**Spark Web UI**
*if running a spark job on local machine then navigate to http://localhost:4040 to check job details and memory consumption*


In [11]:
from pyspark.sql.functions import col,count

# Iterate over each column in the DataFrame
for column in user_df.columns:
    distinct_values = user_df.groupBy(col(column)).agg(count("*").alias("count"))
    distinct_values.show()

+-------------+-----+
|average_stars|count|
+-------------+-----+
|         3.26| 1766|
|         2.86| 3088|
|         1.82|  300|
|         4.19| 4165|
|          2.4| 3049|
|         2.41|  456|
|         2.82| 1572|
|         2.62|  797|
|         4.88| 3416|
|         3.02|  600|
|         4.92| 1770|
|         2.55|  952|
|         4.23| 4157|
|         4.02| 2583|
|         3.08| 2293|
|         4.36| 5107|
|         3.57| 6430|
|         4.93| 1193|
|          3.5|35031|
|         3.56| 5360|
+-------------+-----+
only showing top 20 rows

+---------------+-----+
|compliment_cool|count|
+---------------+-----+
|             26|  582|
|             29|  557|
|           3506|    1|
|           2250|    2|
|             65|  135|
|            541|    2|
|            418|    6|
|            191|   12|
|           1258|    3|
|            270|    7|
|            222|   16|
|           1409|    1|
|            938|    2|
|            730|    3|
|            243|   12|
|            2

+----+-------+
|fans|  count|
+----+-------+
|  29|    668|
|  26|    899|
|  65|    138|
| 191|     12|
| 558|      2|
| 270|      7|
|1532|      1|
| 720|      1|
| 243|      2|
| 442|      4|
| 278|      2|
|  19|   1515|
|  54|    205|
|   0|1577870|
| 348|      3|
| 287|      2|
| 112|     40|
| 113|     36|
| 155|     21|
| 167|     20|
+----+-------+
only showing top 20 rows

+--------------------+-----+
|             friends|count|
+--------------------+-----+
|h4ZqDUqVcektL45xG...|    1|
|xyFtRXaTjhStZB3Zb...|    1|
|HzoQKKHDq9BI37dyJ...|    1|
|OqkxLARP6t-IPrYpY...|    1|
|o9ecuikSpMbjMhhQ_...|    1|
|cjqpThXUJHDWXzrA8...|    1|
|SWLtQLehJ80wkcSWs...|    1|
|6nJhLiNjvb34bD84e...|    1|
|SP990dMcmW5OTbgEg...|    1|
|GKUdXdjMuyZG1Iwsv...|    1|
|_RjmMyRFp8EpbNAWH...|    1|
|DtnH690w5H1Yb681e...|    1|
|NIfobhb9x7nWj36eg...|    1|
|ncQ34aegMiUxxsuNS...|    1|
|K2VZMLykxnctDXWgO...|    1|
|eOLIUE7Noqku_8bxz...|    1|
|PS5moRBxktcWZJf0y...|    1|
|3zxy3LVBV3ttxoYbY...|    1|
|yCC7

In [12]:
import time
# Start the timer
start_time = time.time()
#Missing values
from pyspark.sql.functions import col, isnull

# Missing values
total_rows = user_df.count()
for i in range(len(user_df.columns)):
    column_name = user_df.columns[i]
    n = user_df.filter(isnull(col(column_name))).count()
    p = (n / total_rows) * 100
    print(f"Missing values in {column_name}: {p:.2f}%",' and total count: ',n)

# Stop the timer
end_time = time.time()

# Calculate the execution time
execution_time = end_time - start_time

print('\n')

# Print the execution time
print("Execution Time:", execution_time, "seconds")

Missing values in average_stars: 0.00%  and total count:  0
Missing values in compliment_cool: 0.00%  and total count:  0
Missing values in compliment_cute: 0.00%  and total count:  0
Missing values in compliment_funny: 0.00%  and total count:  0
Missing values in compliment_hot: 0.00%  and total count:  0
Missing values in compliment_list: 0.00%  and total count:  0
Missing values in compliment_more: 0.00%  and total count:  0
Missing values in compliment_note: 0.00%  and total count:  0
Missing values in compliment_photos: 0.00%  and total count:  0
Missing values in compliment_plain: 0.00%  and total count:  0
Missing values in compliment_profile: 0.00%  and total count:  0
Missing values in compliment_writer: 0.00%  and total count:  0
Missing values in cool: 0.00%  and total count:  0
Missing values in elite: 0.00%  and total count:  0
Missing values in fans: 0.00%  and total count:  0
Missing values in friends: 0.00%  and total count:  0
Missing values in funny: 0.00%  and total 

In [5]:
# Register the DataFrame as a temporary table
user_df.createOrReplaceTempView("user_table")

In [6]:
# Enable cost-based optimization
spark.conf.set("spark.sql.cbo.enabled", "true")
# Enable code generation
spark.conf.set("spark.sql.codegen.wholeStage", "true")

In [15]:
import time
# Start the timer
start_time = time.time()
result=spark.sql('''select * from 
(select user_id,
row_number() over (partition by average_stars,
compliment_cool,
compliment_cute,
compliment_funny,
compliment_hot,
compliment_list,
compliment_more,
compliment_note,
compliment_photos,
compliment_plain,
compliment_profile,
compliment_writer,
cool,
elite,
fans,
friends,
funny,
name,
review_count,
useful,
yelping_since
order by 
average_stars,
compliment_cool,
compliment_cute,
compliment_funny,
compliment_hot,
compliment_list,
compliment_more,
compliment_note,
compliment_photos,
compliment_plain,
compliment_profile,
compliment_writer,
cool,
elite,
fans,
friends,
funny,
name,
review_count,
useful,
yelping_since) as rown
from user_table )
where rown>2
''')
result.show()

# Stop the timer
end_time = time.time()

# Calculate the execution time
execution_time = end_time - start_time

print('\n')

# Print the execution time
print("Execution Time:", execution_time, "seconds")

+-------+----+
|user_id|rown|
+-------+----+
+-------+----+



Execution Time: 20.147220611572266 seconds


In [16]:
result.explain(mode='cost')
# mode= 'formatted'
#mode='codegen'
#mode='extended'

== Optimized Logical Plan ==
Project [user_id#27, rown#3554], Statistics(sizeInBytes=420.7 MiB)
+- Filter (rown#3554 > 2), Statistics(sizeInBytes=3.2 GiB)
   +- Window [row_number() windowspecdefinition(average_stars#7, compliment_cool#8L, compliment_cute#9L, compliment_funny#10L, compliment_hot#11L, compliment_list#12L, compliment_more#13L, compliment_note#14L, compliment_photos#15L, compliment_plain#16L, compliment_profile#17L, compliment_writer#18L, cool#19L, elite#20, fans#21L, friends#22, funny#23L, name#24, review_count#25L, useful#26L, yelping_since#28, average_stars#7 ASC NULLS FIRST, compliment_cool#8L ASC NULLS FIRST, compliment_cute#9L ASC NULLS FIRST, ... 19 more fields) AS rown#3554], [knownfloatingpointnormalized(normalizenanandzero(average_stars#7)), compliment_cool#8L, compliment_cute#9L, compliment_funny#10L, compliment_hot#11L, compliment_list#12L, compliment_more#13L, compliment_note#14L, compliment_photos#15L, compliment_plain#16L, compliment_profile#17L, compliment

In [8]:
import time
# Start the timer
start_time = time.time()
result=spark.sql(
'''
select count(user_id) as cnt
from user_table
group by
average_stars,
compliment_cool,
compliment_cute,
compliment_funny,
compliment_hot,
compliment_list,
compliment_more,
compliment_note,
compliment_photos,
compliment_plain,
compliment_profile,
compliment_writer,
cool,
elite,
fans,
friends,
funny,
name,
review_count,
useful,
yelping_since
having count(user_id)>1
'''
)

result.show()

# Stop the timer
end_time = time.time()

# Calculate the execution time
execution_time = end_time - start_time

print('\n')

# Print the execution time
print("Execution Time:", execution_time, "seconds")

+---+
|cnt|
+---+
+---+



Execution Time: 13.220051527023315 seconds


In [18]:
result.explain(mode='cost')

== Optimized Logical Plan ==
Filter (cnt#3589L > 1), Statistics(sizeInBytes=210.3 MiB)
+- Aggregate [average_stars#7, compliment_cool#8L, compliment_cute#9L, compliment_funny#10L, compliment_hot#11L, compliment_list#12L, compliment_more#13L, compliment_note#14L, compliment_photos#15L, compliment_plain#16L, compliment_profile#17L, compliment_writer#18L, cool#19L, elite#20, fans#21L, friends#22, funny#23L, name#24, review_count#25L, useful#26L, yelping_since#28], [count(user_id#27) AS cnt#3589L], Statistics(sizeInBytes=210.3 MiB)
   +- Relation [average_stars#7,compliment_cool#8L,compliment_cute#9L,compliment_funny#10L,compliment_hot#11L,compliment_list#12L,compliment_more#13L,compliment_note#14L,compliment_photos#15L,compliment_plain#16L,compliment_profile#17L,compliment_writer#18L,cool#19L,elite#20,fans#21L,friends#22,funny#23L,name#24,review_count#25L,useful#26L,user_id#27,yelping_since#28] json, Statistics(sizeInBytes=3.1 GiB)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false


In [19]:
start=time.time()
#Top user with highest Review count
result=spark.sql(
'''
select u.user_id,u.review_count
from user_table u
where u.review_count=(select max(u1.review_count) from user_table u1)
'''
)
result.show(truncate=False)
end=time.time()
print("Execution time: ",end-start,'seconds')

+----------------------+------------+
|user_id               |review_count|
+----------------------+------------+
|Hi10sGSZNxQH3NLyWSZ1oA|17473       |
+----------------------+------------+

Execution time:  3.213028907775879 seconds


In [20]:
start=time.time()
#Top user with lowest Review count
result=spark.sql(
'''
select u.user_id,u.review_count
from user_table u
where u.review_count=(select min(u1.review_count) from user_table u1)
'''
)
result.show(truncate=False)
end=time.time()
print("Execution time: ",end-start,'seconds')

+----------------------+------------+
|user_id               |review_count|
+----------------------+------------+
|1zEwvAm5TqJh7eqYShNVQA|0           |
|T8qn_FUg24LsdPaax4plNA|0           |
|Pflag9DDmtagR6a-2Z8Jmw|0           |
|MfQc-rOY5Qu_uRcoXDNzKA|0           |
|OSSYJJIGVr53Tk61ZycITQ|0           |
|SfhWhnf-TuTjEulNwhVasA|0           |
|7IBB8V7G-2d4pWdxySAgew|0           |
|h2xeEEOpFyZHBnezJy1pYQ|0           |
|889Zs1Xyz6kucIShNdCJjg|0           |
|ognrtkoN_v890L70KtnkcQ|0           |
|YHceEmeft2GZXpGrVApXQQ|0           |
|STCJB07I94MN34hOzovcZQ|0           |
|q7H2tz-o2Zjnl8mJHxuL6g|0           |
|AmolTw60VjHqfEAHfjgVDQ|0           |
|LXULQLXqjxG6_B5OOJf-MA|0           |
|IcuZF8KioeG-8uXqR4QqPA|0           |
|cco5VSBNHfow4jfLgjZG8A|0           |
|6Vxh445RzNEgBT6ZTBOGhg|0           |
|ih2hlbMfrdiJrUTrT48fVQ|0           |
|MJ1ht_fX0A8s6IK9Bi4joA|0           |
+----------------------+------------+
only showing top 20 rows

Execution time:  3.0285980701446533 seconds


In [21]:
start=time.time()
#count of users with zero review count
result=spark.sql(
'''
select count(u.user_id),u.review_count
from user_table u
where u.review_count=(select min(u1.review_count) from user_table u1)
group by u.review_count
'''
)
result.show(truncate=False)
end=time.time()
print("Execution time: ",end-start,'seconds')

+--------------+------------+
|count(user_id)|review_count|
+--------------+------------+
|54            |0           |
+--------------+------------+

Execution time:  2.7613346576690674 seconds


In [9]:
start=time.time()
#Top 10 users with more review_count
result=spark.sql(
'''
select u.user_id,u.review_count
from user_table u
order by u.review_count desc
limit 10
'''
)
result.show(truncate=False)
end=time.time()
print("Execution time: ",end-start,'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/top_users_review.csv")

+----------------------+------------+
|user_id               |review_count|
+----------------------+------------+
|Hi10sGSZNxQH3NLyWSZ1oA|17473       |
|8k3aO-mPeyhbR5HUucA5aA|16978       |
|hWDybu_KvYLSdEFzGrniTw|16567       |
|RtGqdDBvvBCjcu5dUqwfzA|12868       |
|P5bUL3Engv-2z6kKohB6qQ|9941        |
|nmdkHL2JKFx55T3nq5VziA|8363        |
|bQCHF5rn5lMI9c5kEwCaNA|8354        |
|8RcEwGrFIgkt9WQ35E6SnQ|7738        |
|Xwnf20FKuikiHcSpcEbpKQ|6766        |
|CxDOIDnH8gp9KXzpBHJYXw|6679        |
+----------------------+------------+

Execution time:  4.838027477264404 seconds


In [23]:
start=time.time()
#top 10 users with least review_count
result=spark.sql(
'''
select count,review_count from (
select count(u.user_id) as count,u.review_count
from user_table u
group by u.review_count
) 
order by review_count
limit 10
'''
)
result.show(truncate=False)
end=time.time()
print("Execution time: ",end-start,'seconds')

+------+------------+
|count |review_count|
+------+------------+
|54    |0           |
|345437|1           |
|243744|2           |
|179060|3           |
|132508|4           |
|102241|5           |
|82433 |6           |
|68033 |7           |
|57627 |8           |
|50241 |9           |
+------+------------+

Execution time:  4.156750440597534 seconds


In [24]:
import time
start = time.time()
# Top 10 users with least review_count
result = spark.sql('''
select CONCAT_WS('; ', collect_list(user_id)) as userid_list,review_count 
from (
SELECT user_id, review_count,
    count(*) OVER (PARTITION BY review_count ORDER BY review_count) AS count
    FROM user_table   
) 
group by review_count
order by review_count
limit 10
''')
result.show()
end = time.time()
print("Execution time: ", end - start, 'seconds')


+--------------------+------------+
|         userid_list|review_count|
+--------------------+------------+
|1zEwvAm5TqJh7eqYS...|           0|
|5Z0qHr3lRAy311-D-...|           1|
|q-kq52Hm8a5ajWJ_d...|           2|
|pxyzkPY6a4FVF0aDj...|           3|
|EWzlzkAYBh4ZYujdy...|           4|
|SSp_Idg6Ge-Lb6T99...|           5|
|kP7Iqhiznm3NyjZ06...|           6|
|EItxyUbSIAMZH_Crg...|           7|
|c77-K8o5-7u25cWxx...|           8|
|1u29fyVKDnupqBhEv...|           9|
+--------------------+------------+

Execution time:  5.918423652648926 seconds


In [10]:
start=time.time()
#Top 10 user names with more review_count
result=spark.sql(
'''
select u.name,u.review_count
from user_table u
order by u.review_count desc
limit 10
'''
)
result.show(truncate=False)
end=time.time()
print("Execution time: ",end-start,'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/top_username_review.csv")

+--------+------------+
|name    |review_count|
+--------+------------+
|Fox     |17473       |
|Victor  |16978       |
|Bruce   |16567       |
|Shila   |12868       |
|Kim     |9941        |
|Nijole  |8363        |
|Vincent |8354        |
|George  |7738        |
|Kenneth |6766        |
|Jennifer|6679        |
+--------+------------+

Execution time:  1.237861156463623 seconds


In [26]:
import time
start = time.time()
# Top 10 users with least review_count
result = spark.sql('''
select CONCAT_WS('; ', collect_list(name)) as users_list,review_count 
from (
SELECT name, review_count,
    count(*) OVER (PARTITION BY review_count ORDER BY review_count) AS count
    FROM user_table   
) 
group by review_count
order by review_count
limit 10
''')
result.show()
end = time.time()
print("Execution time: ", end - start, 'seconds')

+--------------------+------------+
|          users_list|review_count|
+--------------------+------------+
|Bre; Bella; Jazmi...|           0|
|charles; Meg; Pau...|           1|
|Minji; Audrey; Co...|           2|
|Dana; Ferdie; Joe...|           3|
|Eric; Katie; Carl...|           4|
|Michael; Felix; T...|           5|
|Jenna; Kris; Dais...|           6|
|Andy; Michael; Cr...|           7|
|Pete; David; Juli...|           8|
|Sean; Tim; Ilo; A...|           9|
+--------------------+------------+

Execution time:  4.3691675662994385 seconds


In [11]:
import time
start = time.time()
# month in which more number of users joined yelp
result = spark.sql('''
select count,month from
(
SELECT count(user_id) as count, date_format(yelping_since, 'yyyy') AS month
FROM user_table
group by  date_format(yelping_since, 'yyyy') 
)
order by count desc
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/month_more_users.csv")

+------+-----+
|count |month|
+------+-----+
|247850|2015 |
|233465|2014 |
|217620|2016 |
|209762|2013 |
|195955|2012 |
|176435|2011 |
|151024|2017 |
|133568|2018 |
|109054|2010 |
|104655|2019 |
|64911 |2009 |
|47444 |2020 |
|40485 |2021 |
|31097 |2008 |
|15340 |2007 |
|5423  |2006 |
|2782  |2022 |
|937   |2005 |
|90    |2004 |
+------+-----+

Execution time:  1.9785974025726318 seconds


In [28]:
import time
start = time.time()
# which month has raise and fall of users
result = spark.sql('''
select count,month from
(
SELECT count(user_id) as count, date_format(yelping_since, 'yyyy') AS month
FROM user_table
group by  date_format(yelping_since, 'yyyy') 
)
order by month
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')

+------+-----+
|count |month|
+------+-----+
|90    |2004 |
|937   |2005 |
|5423  |2006 |
|15340 |2007 |
|31097 |2008 |
|64911 |2009 |
|109054|2010 |
|176435|2011 |
|195955|2012 |
|209762|2013 |
|233465|2014 |
|247850|2015 |
|217620|2016 |
|151024|2017 |
|133568|2018 |
|104655|2019 |
|47444 |2020 |
|40485 |2021 |
|2782  |2022 |
+------+-----+

Execution time:  1.8021900653839111 seconds


In [12]:
#which year has highest review count
import time
start = time.time()
result = spark.sql('''
select count,month from
(
SELECT sum(review_count) as count, date_format(yelping_since, 'yyyy') AS month
FROM user_table
group by  date_format(yelping_since, 'yyyy') 
)
order by count desc
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/month_more_review.csv")

+-------+-----+
|count  |month|
+-------+-----+
|6344607|2011 |
|5444361|2010 |
|5405586|2012 |
|4820434|2013 |
|4401364|2009 |
|4354298|2014 |
|3630507|2015 |
|3162898|2008 |
|2500839|2016 |
|2011487|2007 |
|1457654|2017 |
|938027 |2018 |
|920176 |2006 |
|559500 |2019 |
|226005 |2005 |
|197284 |2020 |
|112349 |2021 |
|14577  |2004 |
|3723   |2022 |
+-------+-----+

Execution time:  1.5910356044769287 seconds


In [13]:
import time
start = time.time()
# top 10 users with max friends
result = spark.sql('''
select * from (
SELECT user_id,name,size(split(friends, ',')) as total_friends_count
FROM user_table
) 
order by total_friends_count desc
limit 10
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/top_users_frnds.csv")

+----------------------+------+-------------------+
|user_id               |name  |total_friends_count|
+----------------------+------+-------------------+
|qVc8ODYU5SZjKXVBgXdI7w|Walker|14995              |
|iLjMdZi0Tm7DQxX1C1_2dg|Ruggy |12395              |
|ZIOCmdFaMIF56FR-nWr_2A|Randy |11026              |
|mV4lknblF-zOKSF8nlGqDA|Scott |10366              |
|Oi1qbcz2m2SnwUeztGYcnQ|Steven|10072              |
|hizGc5W1tBHPghM5YKCAtg|Katie |9390               |
|IU86PZPgTDCFwJEuAg2j7g|Danny |9217               |
|n4Y6wdh6om3QtIFlWakfDQ|Rob   |8989               |
|djxnI8Ux8ZYQJhiOQkrRhA|Abby  |8858               |
|F_5_UNX-wrAFCXuAkBZRDw|Rodney|8809               |
+----------------------+------+-------------------+

Execution time:  2.5419836044311523 seconds


In [31]:
import time
start = time.time()
# top 10 users with less friends
result = spark.sql('''
select * from (
SELECT user_id,name,size(split(friends, ',')) as total_friends_count
FROM user_table
) 
order by total_friends_count 
limit 10
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')

+----------------------+-------------+-------------------+
|user_id               |name         |total_friends_count|
+----------------------+-------------+-------------------+
|3wmhky2VOA394cQrBwaCLg|John         |1                  |
|nwhkfR74Cinn5ltS4WIOkQ|Houssam      |1                  |
|dRO1O-OxJ7EwjBm0deyuig|Gary         |1                  |
|n8bbgZJLENE4dC4E743ukQ|Caithlin     |1                  |
|InYZY1kzw3IkffkdyBDd4w|Kelly        |1                  |
|FiFC3I3B4RZKbbQS4v45wg|Tracey       |1                  |
|M7kB6fibEsDgo6XsDkG0cA|Katey        |1                  |
|YI6PXqtVQLTOxtBemBTkcQ|Irving       |1                  |
|VapsfRxmf7bs5IzGLh5smA|Randy        |1                  |
|gBc8GV605lzhVzr3ki0wbA|Ms. Austridge|1                  |
+----------------------+-------------+-------------------+

Execution time:  2.4896042346954346 seconds


In [32]:
import time
start = time.time()
# top 10 users with less friends
result = spark.sql('''
select concat_ws(',',collect_list(user_id)) as userid_list,
concat_ws(',',collect_list(name)) as name_list,
total_friends_count
from (
SELECT user_id,name,size(split(friends, ',')) as total_friends_count
FROM user_table
) 
group by total_friends_count
order by total_friends_count 
limit 10
''')
result.show()
end = time.time()
print("Execution time: ", end - start, 'seconds')

+--------------------+--------------------+-------------------+
|         userid_list|           name_list|total_friends_count|
+--------------------+--------------------+-------------------+
|Je6-vkrCJxex_cmRE...|Lea,picunte,Nate,...|                  1|
|I1UOCUGCd3STY-PYe...|SilverGirl,Pradee...|                  2|
|PXL1N5xd-CVjw9bFw...|Carolyn,Greg,Kate...|                  3|
|cEHn8OkDjjWQYbMKa...|Jesse,Rob,David,T...|                  4|
|XmqE8z4cGfCN5p3pl...|Tessa,Jon,Anastas...|                  5|
|EufrlD8h-0Q-ffHs5...|Joe,Molly,Nathan,...|                  6|
|GJrPKaJtBtz2GwF7p...|Katelyn,Johm,Tony...|                  7|
|NzJ3cF6aqdbe6aUA2...|Tim,Ashley,Judy,W...|                  8|
|7YkIRLA7pErLCV637...|Mark,Amy,Larisa,B...|                  9|
|ksp0lKhCvwKOj8PS7...|Gisselle,Jand,Sri...|                 10|
+--------------------+--------------------+-------------------+

Execution time:  7.193603754043579 seconds


In [33]:
import time
start = time.time()
# top 10 users with less friends
result = spark.sql('''
SELECT userid_list, 
name_list, 
size(split(userid_list, ',')) as userid_cnt,
size(split(name_list, ',')) as name_cnt,
total_friends_count
    FROM (
        SELECT concat_ws(',', collect_list(user_id)) as userid_list,
               concat_ws(',', collect_list(name)) as name_list,
               total_friends_count
        FROM (
            SELECT user_id, name, size(split(friends, ',')) as total_friends_count
            FROM user_table
        ) 
        GROUP BY total_friends_count
        ORDER BY total_friends_count 
        LIMIT 10
    )
''')
result.show()
end = time.time()
print("Execution time: ", end - start, 'seconds')

+--------------------+--------------------+----------+--------+-------------------+
|         userid_list|           name_list|userid_cnt|name_cnt|total_friends_count|
+--------------------+--------------------+----------+--------+-------------------+
|Je6-vkrCJxex_cmRE...|Lea,picunte,Nate,...|    993654|  993720|                  1|
|I1UOCUGCd3STY-PYe...|SilverGirl,Pradee...|     57923|   57932|                  2|
|PXL1N5xd-CVjw9bFw...|Carolyn,Greg,Kate...|     38827|   38829|                  3|
|cEHn8OkDjjWQYbMKa...|Jesse,Rob,David,T...|     30168|   30168|                  4|
|XmqE8z4cGfCN5p3pl...|Tessa,Jon,Anastas...|     24389|   24393|                  5|
|EufrlD8h-0Q-ffHs5...|Joe,Molly,Nathan,...|     20785|   20786|                  6|
|GJrPKaJtBtz2GwF7p...|Katelyn,Johm,Tony...|     18323|   18325|                  7|
|NzJ3cF6aqdbe6aUA2...|Tim,Ashley,Judy,W...|     16492|   16494|                  8|
|7YkIRLA7pErLCV637...|Mark,Amy,Larisa,B...|     14954|   14954|             

In [14]:
import time
start = time.time()
# users with highest userids
result = spark.sql('''
select * from (
SELECT name,count(user_id) as cnt
FROM user_table
group by name
having count(user_id)>1
) order by cnt desc
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/top_users_ids.csv")

+--------+-----+
|name    |cnt  |
+--------+-----+
|John    |18719|
|Michael |16645|
|David   |15967|
|Chris   |14591|
|Mike    |13390|
|Jennifer|13275|
|Sarah   |11893|
|Jessica |11688|
|Lisa    |10623|
|Michelle|9804 |
|Brian   |9634 |
|Mark    |9510 |
|Ashley  |9031 |
|Amanda  |9013 |
|James   |8647 |
|Robert  |8434 |
|Amy     |8335 |
|Lauren  |8294 |
|Jason   |8291 |
|Matt    |8211 |
+--------+-----+
only showing top 20 rows

Execution time:  4.359771251678467 seconds


In [15]:
import time
start = time.time()
# top 10 users with highest useful votes
result = spark.sql('''
select * from (
SELECT user_id,name,sum(useful) as total_useful_votes
FROM user_table
group by user_id,name
)
order by total_useful_votes desc
limit 10
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/top_users_useful.csv")

+----------------------+--------+------------------+
|user_id               |name    |total_useful_votes|
+----------------------+--------+------------------+
|Hi10sGSZNxQH3NLyWSZ1oA|Fox     |206296            |
|--2vR0DIsmQ6WfcSzKWigw|Harald  |205765            |
|JjXuiru1_ONzDkYVrHN0aw|Richard |183512            |
|lvthTfCQGD0qaEk6jCdRdQ|Ariel   |182788            |
|hWDybu_KvYLSdEFzGrniTw|Bruce   |173089            |
|8k3aO-mPeyhbR5HUucA5aA|Victor  |152979            |
|Kf5PVQUdDvQYf-MXoeWqZA|Marianne|142453            |
|AbMjnKOwg736fcIu8apuyQ|Michael |124311            |
|7W-p-PJlmrzg0mk3pWKoyQ|Arshad  |117066            |
|ax7SnXOTIpatbsmqHLqVow|Rohlin  |112557            |
+----------------------+--------+------------------+

Execution time:  4.823875665664673 seconds


In [36]:
import time
start = time.time()
# top 10 users with lowest useful votes
result = spark.sql('''
select user_list,
size(split(user_list,',')) as user_cnt,
name_list,
size(split(name_list,',')) as name_cnt,
total_useful_votes
from (
select concat_ws(',',collect_list(user_id)) as user_list,
concat_ws(',',collect_list(name)) as name_list,
total_useful_votes
from (
SELECT user_id,name,sum(useful) as total_useful_votes
FROM user_table
group by user_id,name
)
group by total_useful_votes
order by total_useful_votes 
limit 10
)
''')
result.show()
end = time.time()
print("Execution time: ", end - start, 'seconds')

+--------------------+--------+--------------------+--------+------------------+
|           user_list|user_cnt|           name_list|name_cnt|total_useful_votes|
+--------------------+--------+--------------------+--------+------------------+
|b-Ut_nrW497rLuTMz...|  530210|Luis,Julia,Chris,...|  530236|                 0|
|joVrffMWAXIrAPbSq...|  256616|Seth,Lynn,B,Dan,R...|  256626|                 1|
|qCF7Jg_TtrH4cBJMA...|  160299|Joe,Maria,Mark,Da...|  160308|                 2|
|EHt-_kJ5l9IdwdKI5...|  114152|Vanaro,Maggie,Mea...|  114156|                 3|
|8QSyIvO8XxzeVMtoW...|   86669|Katharine,Thomas,...|   86674|                 4|
|oS-v0qBEAWQGgynqY...|   69303|Donna,Richard,Col...|   69308|                 5|
|UVg8BFOEEigQ-rfZk...|   57585|Paula,Justin,Kevi...|   57588|                 6|
|sGapFTUgvIcEXr4yz...|   47808|Mary Pat,Meilee,J...|   47809|                 7|
|c-rh7AvkmI04oi8rS...|   41951|Janelle,Eric,Tom,...|   41953|                 8|
|194MmDTNAdVzxOUJo...|   359

In [16]:
import time
start = time.time()
# top 10 users with highest funny votes
result = spark.sql('''
select * from (
SELECT user_id,name,sum(funny) as total_funny_votes
FROM user_table
group by user_id,name
)
order by total_funny_votes desc
limit 10
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/top_users_funny.csv")

+----------------------+--------+-----------------+
|user_id               |name    |total_funny_votes|
+----------------------+--------+-----------------+
|Hi10sGSZNxQH3NLyWSZ1oA|Fox     |185823           |
|--2vR0DIsmQ6WfcSzKWigw|Harald  |173096           |
|JjXuiru1_ONzDkYVrHN0aw|Richard |129638           |
|ax7SnXOTIpatbsmqHLqVow|Rohlin  |103618           |
|hWDybu_KvYLSdEFzGrniTw|Bruce   |98459            |
|7W-p-PJlmrzg0mk3pWKoyQ|Arshad  |96505            |
|AbMjnKOwg736fcIu8apuyQ|Michael |91842            |
|8k3aO-mPeyhbR5HUucA5aA|Victor  |84992            |
|W7DHyQlY_kXls2iXt-_2Ag|Maggie  |84952            |
|Kf5PVQUdDvQYf-MXoeWqZA|Marianne|81937            |
+----------------------+--------+-----------------+

Execution time:  4.552423477172852 seconds


In [38]:
import time
start = time.time()
# top 10 users with lowest funny votes
result = spark.sql('''
select user_list,
size(split(user_list,',')) as user_cnt,
name_list,
size(split(name_list,',')) as name_cnt,
total_funny_votes
from (
select concat_ws(',',collect_list(user_id)) as user_list,
concat_ws(',',collect_list(name)) as name_list,
total_funny_votes
from (
SELECT user_id,name,sum(funny) as total_funny_votes
FROM user_table
group by user_id,name
)
group by total_funny_votes
order by total_funny_votes 
limit 10
)
''')
result.show()
end = time.time()
print("Execution time: ", end - start, 'seconds')

+--------------------+--------+--------------------+--------+-----------------+
|           user_list|user_cnt|           name_list|name_cnt|total_funny_votes|
+--------------------+--------+--------------------+--------+-----------------+
|p-QoQrhzB6QATSDoT...| 1097703|Lynn,Tameka,M,B,J...| 1097747|                0|
|qyYUfF_vv3m6JVlrG...|  267049|Jason,Colleen,Rac...|  267063|                1|
|JdNyMdgk9Nc7X2K2n...|  132066|Jennifer,Richard,...|  132074|                2|
|D1OqBYZQSEWuzuHPw...|   81402|Andre,Christie,Ch...|   81404|                3|
|TN20uwLnSpXGYCZ9z...|   55765|M,Jennifer,David,...|   55768|                4|
|MVtnAsCEcHsWcQbzh...|   40908|Kirk,Stephen,Jaso...|   40913|                5|
|PlyuK1QO4MjZrAxNL...|   31437|Wendy,James,Micha...|   31439|                6|
|QE8Yvcb96znejEzK2...|   25148|Stephanie,Jeff,Me...|   25150|                7|
|4zOwbpXdL4u3ERVt2...|   20346|Jesse,Alison,Thom...|   20347|                8|
|SfTBI2vNwjpqbi3Sk...|   17429|Scott,Ste

In [17]:
import time
start = time.time()
# top 10 users with highest cool votes
result = spark.sql('''
select * from (
SELECT user_id,name,sum(cool) as total_cool_votes
FROM user_table
group by user_id,name
)
order by total_cool_votes desc
limit 10
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/top_users_cool.csv")

+----------------------+--------+----------------+
|user_id               |name    |total_cool_votes|
+----------------------+--------+----------------+
|--2vR0DIsmQ6WfcSzKWigw|Harald  |199878          |
|Hi10sGSZNxQH3NLyWSZ1oA|Fox     |195814          |
|JjXuiru1_ONzDkYVrHN0aw|Richard |173423          |
|lvthTfCQGD0qaEk6jCdRdQ|Ariel   |162667          |
|hWDybu_KvYLSdEFzGrniTw|Bruce   |144849          |
|Kf5PVQUdDvQYf-MXoeWqZA|Marianne|136166          |
|8k3aO-mPeyhbR5HUucA5aA|Victor  |125516          |
|7W-p-PJlmrzg0mk3pWKoyQ|Arshad  |113371          |
|AbMjnKOwg736fcIu8apuyQ|Michael |113069          |
|Ggx8iUdJ7lsQSqXRuclXtg|Nadine  |108733          |
+----------------------+--------+----------------+

Execution time:  4.4758031368255615 seconds


In [40]:
import time
start = time.time()
# top 10 users with lowest cool votes
result = spark.sql('''
select user_list,
size(split(user_list,',')) as user_cnt,
name_list,
size(split(name_list,',')) as name_cnt,
total_cool_votes
from (
select concat_ws(',',collect_list(user_id)) as user_list,
concat_ws(',',collect_list(name)) as name_list,
total_cool_votes
from (
SELECT user_id,name,sum(cool) as total_cool_votes
FROM user_table
group by user_id,name
)
group by total_cool_votes
order by total_cool_votes 
limit 10
)
''')
result.show()
end = time.time()
print("Execution time: ", end - start, 'seconds')

+--------------------+--------+--------------------+--------+----------------+
|           user_list|user_cnt|           name_list|name_cnt|total_cool_votes|
+--------------------+--------+--------------------+--------+----------------+
|b0sgn185jl481ZUEi...| 1042865|suz,Tricia,Calvin...| 1042918|               0|
|2YXL9DBC2wC8fgShO...|  280748|Richard,Christina...|  280759|               1|
|ox8mzdJtA0qGAf6HS...|  137006|Amy,Kimberly,Brya...|  137011|               2|
|bJxrsAOlZyvYrJ3Sw...|   83950|Laura,Jennifer,An...|   83954|               3|
|TN20uwLnSpXGYCZ9z...|   58096|M,Ryan,Lance,R,Be...|   58098|               4|
|N4m2UsRM3taB1oQA4...|   42831|Doug,Carol,Daniel...|   42832|               5|
|fHZGnb0MCK897Zgoi...|   33026|Aaron,Rachael,Bre...|   33027|               6|
|KZ8rv6ZufXyPEXtlR...|   26104|Bridget,Lucero,hi...|   26105|               7|
|FYQ3lTJGnofprl4Wd...|   21672|Mo',Nate,SOFIA,Al...|   21674|               8|
|1LS-1LbfUnLrftzcu...|   18052|Jennifer,Sheryl,M...|

In [51]:
import time
start = time.time()
result = spark.sql('''
SELECT user_id,name,elite
FROM user_table
where name='Walker'
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')

+----------------------+------+--------------+
|user_id               |name  |elite         |
+----------------------+------+--------------+
|qVc8ODYU5SZjKXVBgXdI7w|Walker|2007          |
|jm1BLd50MzZS-ZZDFx4slQ|Walker|2012,2013,2014|
|WusBgax9cWTAaJYxDL_4mg|Walker|2010          |
|Mzmy5ygEhRM5sdBzKpcW1A|Walker|              |
|0ct4lELffzoE5aWEcSlB8w|Walker|              |
|IMMg03oYVbJ7pL3Q9EQ8UQ|Walker|              |
|STLmQk1p4N4zuoe82zVLzQ|Walker|              |
|aX3THi4MIgZ-zG_MNUFvZg|Walker|              |
|HMPnra3lic-e-RpEpZ8oyw|Walker|              |
|unQpKor6pJ6Oq14Ial8F6Q|Walker|              |
|ktXbKmqq-60zsj7DXBuE3A|Walker|              |
|v3aRkWUPFnu3txcTB93kUw|Walker|              |
|jpfkJQ_ZkrE-3JFLiCm0uA|Walker|              |
|cvktfi2722cb7iRVNu8qdQ|Walker|              |
|RxVYiydKjQ3cWxxyRSXGCQ|Walker|              |
|njVkT6AdF9oqbpdodG_lHg|Walker|              |
|f71odjKb2PiY8Cc1-RWp3Q|Walker|              |
|RbBNGwFFbLdyDE-DdOfG_w|Walker|              |
|KPq3-kzgWjFK

In [18]:
#Function to return number of years a user was active and engaged
def number_of_years(str_inp):
    lst=str_inp.split(',')
    valid_lst=[i for i in lst if len(i)==4]
    if len(valid_lst)>1:
        return int(sorted(valid_lst)[-1])-int(sorted(valid_lst)[0])
    elif len(valid_lst)==1:
        return 1
    else:
        return 0

In [19]:
# converting python function to spark UDF (user defined function)
udf_number_of_years = udf(number_of_years, IntegerType()) #mention function name and its return type

In [20]:
# Registering the function in spark with any name or same name to use in spark SQL
spark.udf.register("udf_number_of_years", udf_number_of_years)

<function __main__.number_of_years(str_inp)>

In [21]:
#users with more number of active years to least number of active years
import time
start = time.time()
result = spark.sql('''
select 
concat_ws(',', collect_list(user_id)) as user_id,
years_active
from (
SELECT 
user_id,
elite,
udf_number_of_years(nvl(elite,' ')) as years_active
FROM user_table
--where name='Walker'
)
group by years_active
order by years_active desc
''')
result.show()
end = time.time()
print("Execution time: ", end - start, 'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/top_users_active_year.csv")

+--------------------+------------+
|             user_id|years_active|
+--------------------+------------+
|NIhcRW6DWvk1JQhDh...|          15|
|hHDltEEGlvuhRjDvf...|          14|
|3zxy3LVBV3ttxoYbY...|          13|
|j14WgRoU_-2ZE1aw1...|          12|
|QF1Kuhs8iwLWANNZx...|          11|
|MGPQVLsODMm9ZtYQW...|          10|
|65uUyG9yuF0rdMh0z...|           9|
|q_QQ5kBBwlCcbL1s4...|           8|
|RgDVC3ZUBqpEe6Y1k...|           7|
|OlJ9vcVFB1iEKcZO-...|           6|
|2l0O1EI1m0yWjFo2z...|           5|
|2WnXYQFK0hXEoTxPt...|           4|
|xoZvMJPDW6Q9pDAXI...|           3|
|SZDeASXq7o05mMNLs...|           2|
|qVc8ODYU5SZjKXVBg...|           1|
|hA5lMy-EnncsH4JoR...|           0|
+--------------------+------------+

Execution time:  13.367419242858887 seconds


In [22]:
import time
start = time.time()
# top 10 users with highest fan following
result = spark.sql('''
select * from (
SELECT name,sum(fans) as total_fans
FROM user_table
group by name
)
order by total_fans desc
limit 10
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/top_users_fan.csv")

+---------+----------+
|name     |total_fans|
+---------+----------+
|Mike     |28137     |
|Jennifer |25423     |
|Michael  |23140     |
|Michelle |20859     |
|Jessica  |20412     |
|John     |20236     |
|David    |18076     |
|Stephanie|16123     |
|Amy      |16038     |
|Chris    |15851     |
+---------+----------+

Execution time:  4.520035743713379 seconds


In [70]:
import time
start = time.time()
# top 10 users with least fan following
result = spark.sql('''
select concat_ws(',',collect_list(name)) as names,
total_fans
from (
SELECT name,sum(fans) as total_fans
FROM user_table
group by name
)
group by total_fans
order by total_fans 
limit 10
''')
result.show()
end = time.time()
print("Execution time: ", end - start, 'seconds')

+--------------------+----------+
|               names|total_fans|
+--------------------+----------+
|Pamlyn,T. Antonio...|         0|
|Ahamed,Chenai,Ron...|         1|
|Nuke,MARSHA,Anina...|         2|
|Shaundel,Arkay,Be...|         3|
|Goldenpig,Brieann...|         4|
|Statia,Vipul,Ljub...|         5|
|Pavee,deborah,tex...|         6|
|Jonina,SouthPhill...|         7|
|Omaha,Grace Joy,E...|         8|
|Han Nah,Pomy,Lern...|         9|
+--------------------+----------+

Execution time:  4.939344882965088 seconds


In [83]:
import time
start = time.time()
result = spark.sql('''
select * from (
SELECT name,average_stars 
FROM user_table
where name='Walker'
)
''')
result.show(truncate=False)
end = time.time()
print("Execution time: ", end - start, 'seconds')

+------+-------------+
|name  |average_stars|
+------+-------------+
|Walker|3.91         |
|Walker|3.49         |
|Walker|4.2          |
|Walker|4.71         |
|Walker|4.22         |
|Walker|3.95         |
|Walker|2.25         |
|Walker|2.33         |
|Walker|1.0          |
|Walker|2.33         |
|Walker|5.0          |
|Walker|2.0          |
|Walker|5.0          |
|Walker|1.0          |
|Walker|5.0          |
|Walker|3.0          |
|Walker|5.0          |
|Walker|5.0          |
|Walker|1.0          |
|Walker|4.5          |
+------+-------------+
only showing top 20 rows

Execution time:  0.8407785892486572 seconds


In [23]:
# Max star obtained by a user, rounding off to 5,4,3,2,1
import time
start = time.time()
result = spark.sql('''
select concat_ws(',',collect_list(name)) as names,
max_star
from (
SELECT name,round(max(average_stars),0) as max_star
FROM user_table
--where name='Walker'
group by name
)
group by max_star
order by max_star desc
''')
result.show()
end = time.time()
print("Execution time: ", end - start, 'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/top_users_max_star.csv")

+--------------------+--------+
|               names|max_star|
+--------------------+--------+
|K,Laurence,Tyler,...|     5.0|
|Zuzzi,Zlatko,Hide...|     4.0|
|A. Brigitte,Djaz,...|     3.0|
|ProudPelican,Star...|     2.0|
|Frank-The,Chinthi...|     1.0|
+--------------------+--------+

Execution time:  4.828467130661011 seconds


In [24]:
import time
start = time.time()
result = spark.sql('''
select 
concat_ws(',',collect_list(name)) as names,
total_comp_hot,
total_comp_more,
total_comp_profile,
total_comp_cute,
total_comp_list,
total_comp_note,
total_comp_plain,
total_comp_cool,
total_comp_funny,
total_comp_writer,
total_comp_photos
from (
SELECT name,
sum(compliment_hot) as total_comp_hot,
sum(compliment_more) as total_comp_more,
sum(compliment_profile) as total_comp_profile,
sum(compliment_cute) as total_comp_cute,
sum(compliment_list) as total_comp_list,
sum(compliment_note) as total_comp_note,
sum(compliment_plain) as total_comp_plain,
sum(compliment_cool) as total_comp_cool,
sum(compliment_funny) as total_comp_funny,
sum(compliment_writer) as total_comp_writer,
sum(compliment_photos) as total_comp_photos
FROM user_table
group by name
)
group by 
total_comp_hot,
total_comp_more,
total_comp_profile,
total_comp_cute,
total_comp_list,
total_comp_note,
total_comp_plain,
total_comp_cool,
total_comp_funny,
total_comp_writer,
total_comp_photos
order by names  
''')
result.show()
end = time.time()
print("Execution time: ", end - start, 'seconds')
result.write.option("header",True).csv("C:/Supriyaa-spark-notes/tableau files/User/top_users_compliment.csv")

+--------------------+--------------+---------------+------------------+---------------+---------------+---------------+----------------+---------------+----------------+-----------------+-----------------+
|               names|total_comp_hot|total_comp_more|total_comp_profile|total_comp_cute|total_comp_list|total_comp_note|total_comp_plain|total_comp_cool|total_comp_funny|total_comp_writer|total_comp_photos|
+--------------------+--------------+---------------+------------------+---------------+---------------+---------------+----------------+---------------+----------------+-----------------+-----------------+
| Kurt,shey,Karden...|             0|              0|                 0|              1|              0|              0|               0|              0|               0|                1|                0|
|              'Danny|             2|              3|                 1|              0|              0|              6|               9|              2|               2|  